In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np

df = pd.read_csv('./data_without_extra_features/preprocessed_SEFACED.csv')

df['cleaned_text'] = df['cleaned_text'].astype(str)
X = df['cleaned_text']
y = df['Class_Label']

# Convert labels to numerical values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Tokenize the text
max_words = 13000 
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
X_seq = tokenizer.texts_to_sequences(X)

# Define maximum sequence length
maxlen = 100

# Pad sequences to the same length
X_pad = pad_sequences(X_seq, maxlen=maxlen)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pad, y_encoded, test_size=0.2, random_state=42)

# Define CNN model
embedding_dim = 100
num_filters = 128
kernel_size = 5

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=maxlen))
model.add(Conv1D(filters=num_filters, kernel_size=kernel_size, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

# Save the trained model
y_pred_prob = model.predict(X_test)
y_pred = np.where(y_pred_prob > 0.5, 1, 0)

print(classification_report(y_test, y_pred))


Epoch 1/10
441/441 [==============================] - 22s 46ms/step - loss: 0.1335 - accuracy: 0.9384 - val_loss: 0.0320 - val_accuracy: 0.9895
Epoch 2/10
441/441 [==============================] - 19s 43ms/step - loss: 0.0152 - accuracy: 0.9954 - val_loss: 0.0321 - val_accuracy: 0.9889
Epoch 3/10
441/441 [==============================] - 22s 49ms/step - loss: 0.0088 - accuracy: 0.9979 - val_loss: 0.0389 - val_accuracy: 0.9892
Epoch 4/10
441/441 [==============================] - 20s 45ms/step - loss: 0.0070 - accuracy: 0.9982 - val_loss: 0.0420 - val_accuracy: 0.9901
Epoch 5/10
441/441 [==============================] - 21s 48ms/step - loss: 0.0069 - accuracy: 0.9982 - val_loss: 0.0456 - val_accuracy: 0.9912
Epoch 6/10
441/441 [==============================] - 22s 51ms/step - loss: 0.0057 - accuracy: 0.9984 - val_loss: 0.0571 - val_accuracy: 0.9901
Epoch 7/10
441/441 [==============================] - 22s 50ms/step - loss: 0.0062 - accuracy: 0.9984 - val_loss: 0.0521 - val_accuracy:

In [8]:
print(X_pad)
print(y_encoded)

[[    0     0     0 ...    48  5886    15]
 [    0     0     0 ...   123    78 11331]
 [    0     0     0 ...    93    15   123]
 ...
 [    0     0     0 ...    47  1519  3257]
 [    0     0     0 ...   159  1460  5880]
 [    0     0     0 ...  2865   898  4696]]
[1 1 1 ... 0 0 0]
